In [ ]:
# Import necessary libraries
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Step 1: Upload the Excel file from local system
uploaded = files.upload()

Saving Stocks-M.xlsx to Stocks-M.xlsx


In [ ]:
# Step 2: Load the Excel file
file_path = list(uploaded.keys())[0]  # Automatically get the uploaded file name
xls = pd.ExcelFile(file_path)

In [ ]:
# Step 3: Initialize dictionaries to store value and momentum data for each ticker
value_data = {}
momentum_data = {}

# Identify tickers and load corresponding sheets
for sheet_name in xls.sheet_names:
    ticker = sheet_name.split('-')[0]
    if '-V' in sheet_name:
        value_data[ticker] = xls.parse(sheet_name)
    elif '-M' in sheet_name:
        momentum_data[ticker] = xls.parse(sheet_name)

In [ ]:
def calculate_momentum_indicators(momentum_df):
    # Assuming 'Close Price' is in momentum_data[ticker]
    momentum_df['52-Week High'] = momentum_df['Close'].rolling(window=252).max()
    momentum_df['10-Year Return'] = momentum_df['Close'].pct_change(periods=10*252).mean()
    momentum_df['5-Year Return'] = momentum_df['Close'].pct_change(periods=5*252).mean()
    momentum_df['3-Year Return'] = momentum_df['Close'].pct_change(periods=3*252).mean()
    momentum_df['12-Month Return'] = momentum_df['Close'].pct_change(periods=252)
    momentum_df['6-Month Return'] = momentum_df['Close'].pct_change(periods=126)
    momentum_df['50-Day MA'] = momentum_df['Close'].rolling(window=50).mean()
    momentum_df['200-Day MA'] = momentum_df['Close'].rolling(window=200).mean()
    momentum_df['Acceleration Rate'] = momentum_df['50-Day MA'] / momentum_df['200-Day MA']
    return momentum_df[['52-Week High', '10-Year Return', '5-Year Return', '3-Year Return', '12-Month Return', '6-Month Return', 'Acceleration Rate']]

In [ ]:
def download_momentum_indicators(momentum_data):
     with ExcelWriter('momentum_indicators.xlsx') as writer:
         for ticker in momentum_data.keys():
             momentum_indicators = calculate_momentum_indicators(momentum_data[ticker])
             momentum_indicators.to_excel(writer, sheet_name=ticker, index=False) # Set index=False to avoid writing index

In [ ]:
def form_portfolios(momentum_data):
    portfolios = {'momentum': []}
    momentum_scores = [] # Initialize a list to store momentum scores
    tickers_list = [] # Initialize a list to store tickers

    for ticker in momentum_data.keys():
        momentum_indicators = calculate_momentum_indicators(momentum_data[ticker])

        # Calculate a single momentum score (e.g., average of indicators)
        momentum_score = momentum_indicators.mean(axis=1).mean() # Example using mean
        momentum_scores.append(momentum_score)
        tickers_list.append(ticker)
        portfolios['momentum'].append((ticker, momentum_score))


    # Form quintiles using pd.qcut
    quintiles = pd.qcut(momentum_scores, 5, labels=False)
    quintile_data = pd.DataFrame({'Ticker': tickers_list, 'Quintile': quintiles})

    return quintile_data # Or return whatever data structure is needed

In [ ]:
# Step 6: Evaluate Portfolio Performance
def evaluate_portfolio_performance(portfolio):
    # Placeholder for performance evaluation metrics
    performance_metrics = {
        'return': [],
        'sharpe_ratio': [],
        'jensen_alpha': []
    }
    # Iterate over the rows of the DataFrame using iterrows()
    for index, row in portfolio.iterrows():
        ticker = row['Ticker']  # Access the 'Ticker' column
        # score = row['Score']  # Access other columns if needed
        # Assume we have returns data, calculate metrics
        returns = momentum_data[ticker]['Close'].pct_change()
        mean_return = returns.mean() * 252
        std_dev = returns.std() * np.sqrt(252)
        sharpe_ratio = mean_return / std_dev
        # Jensen alpha can be calculated using CAPM if benchmark data is available
        # Placeholder: jensen_alpha = calculate_jensen_alpha(returns, benchmark_returns)

        performance_metrics['return'].append(mean_return)
        performance_metrics['sharpe_ratio'].append(sharpe_ratio)
        # performance_metrics['jensen_alpha'].append(jensen_alpha)

    return performance_metrics



In [ ]:
# Step 7: Run the full analysis
momentum_portfolio = form_portfolios(momentum_data)
momentum_performance = evaluate_portfolio_performance(momentum_portfolio)

# Extract ticker symbols and quintiles from momentum_portfolio
tickers = momentum_portfolio['Ticker'].tolist()
quintiles = momentum_portfolio['Quintile'].tolist() # Extract quintiles

# Prepare data for display in tabular form
momentum_performance_df = pd.DataFrame({
    'Ticker': tickers,  # Use the tickers list
    'Quintile': quintiles,  # Add quintiles to the DataFrame
    'Return': momentum_performance['return'],
    'Sharpe Ratio': momentum_performance['sharpe_ratio']
})

# Sort the DataFrame by 'Quintile' in descending order
momentum_performance_df = momentum_performance_df.sort_values(by='Quintile', ascending=False)

# Display the results in a DataFrame format
print("Momentum Portfolio Performance:")
print(momentum_performance_df)

Momentum Portfolio Performance:
   Ticker  Quintile    Return  Sharpe Ratio
19   NVDA         4  0.641028      1.372286
18   NFLX         4  0.348448      0.797435
16   META         4  0.291693      0.777515
9    BIIB         4  0.058729      0.140404
5     JNJ         3  0.071299      0.401454
7     PEP         3  0.084786      0.467177
11   BIDU         3  0.047868      0.108288
12   MSFT         3  0.264903      1.001170
17   AMZN         2  0.276087      0.844366
6      PG         2  0.088436      0.491030
15    AMD         2  0.496729      0.869789
14   AAPL         2  0.273614      0.980988
13  GOOGL         1  0.221045      0.793290
10    AEP         1  0.095094      0.463482
1      ED         1  0.078907      0.386259
2     DUK         1  0.070355      0.349660
8      KO         0  0.066451      0.378556
4      CL         0  0.059225      0.320056
3     GIS         0  0.052904      0.258553
0      SO         0  0.094594      0.446635
